# Build Predictive Model for Titanic Case

In [3]:
import pandas as pd
import os
import numpy as np

In [4]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path,'train.csv')
test_file_path = os.path.join(processed_data_path,'test.csv')

train_df = pd.read_csv(train_file_path,index_col='PassengerId')
test_df = pd.read_csv(test_file_path,index_col='PassengerId')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
Survived              891 non-null int64
Age                   891 non-null float64
Fare                  891 non-null float64
FamilySize            891 non-null int64
IsMother              891 non-null int64
IsMale                891 non-null int64
Deck_A                891 non-null float64
Deck_B                891 non-null float64
Deck_C                891 non-null float64
Deck_D                891 non-null float64
Deck_E                891 non-null float64
Deck_F                891 non-null float64
Deck_G                891 non-null float64
Deck_Z                891 non-null float64
Pclass_1              891 non-null float64
Pclass_2              891 non-null float64
Pclass_3              891 non-null float64
Title_Lady            891 non-null float64
Title_Master          891 non-null float64
Title_Miss            891 non-null float64
Title_Mr              891 non-null float64


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null float64
Deck_B                418 non-null float64
Deck_C                418 non-null float64
Deck_D                418 non-null float64
Deck_E                418 non-null float64
Deck_F                418 non-null float64
Deck_G                418 non-null float64
Deck_Z                418 non-null float64
Pclass_1              418 non-null float64
Pclass_2              418 non-null float64
Pclass_3              418 non-null float64
Title_Lady            418 non-null float64
Title_Master          418 non-null float64
Title_Miss            418 non-null float64
Title_Mr              418 non-null float64
Title_Mrs             418 non-null flo

## Data Preparation

In [7]:
# X => inputs or predictors; y => output or response
# extract all columns from Age onwards, convert to matrix and change all types to float
X = train_df.loc[:,'Age':].as_matrix().astype('float')
y = train_df['Survived'].ravel()
print (X.shape, y.shape)

(891, 32) (891,)


In [8]:
# train test split
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [9]:
# avg survival in train and test
print('mean survival in train : {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test : {0:.3f}'.format(np.mean(y_test)))

mean survival in train : 0.383
mean survival in test : 0.385


## Baseline Model

In [10]:
from sklearn.dummy import DummyClassifier

In [11]:
# create model
model_dummy = DummyClassifier(strategy='most_frequent',random_state=0)

In [12]:
#train model
model_dummy.fit(X_train,y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [13]:
print('score for baseline model: {0:.2f}'.format(model_dummy.score(X_test,y_test)))

score for baseline model: 0.61


In [14]:
# performance metrics
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

In [15]:
# accuracy score
print('accuracy for baseline model: {0:.2f}'.format(accuracy_score(y_test,model_dummy.predict(X_test))))

accuracy for baseline model: 0.61


In [16]:
# confusion matrix
print('confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test,model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [17]:
# precision and recall scores
print('precision for baseline model: {0:.2f}'.format(precision_score(y_test,model_dummy.predict(X_test))))
print('recall for baseline model: {0:.2f}'.format(recall_score(y_test,model_dummy.predict(X_test))))

precision for baseline model: 0.00
recall for baseline model: 0.00


/Users/kselvadurai/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Submission script

In [18]:
def get_submission_file(model,filename):
    # convert to matrix
    test_X = test_df.as_matrix().astype('float')
    # make predictions
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({'PassengerId': test_df.index,'Survived': predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir,'data','external')
    submission_file_path = os.path.join(submission_data_path,filename)
    # write to file
    df_submission.to_csv(submission_file_path,index=False)

In [19]:
get_submission_file(model_dummy,'01_dummy.csv')

## Logistic Regression Model

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
# create model
model_lr_1 = LogisticRegression(random_state=0)

In [22]:
# train model
model_lr_1.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
# evaluate model
print('score for logistic regression - version 1: {0:.2f}'.format(model_lr_1.score(X_test,y_test)))

score for logistic regression - version 1: 0.83


In [24]:
# create submission file
get_submission_file(model_lr_1,'02_lr.csv')

## Hyperparamter Optimisation

In [25]:
# base model
model_lr = LogisticRegression(random_state=0)

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3)

In [28]:
clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [29]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [30]:
print('best score: {0:.2f}'.format(clf.best_score_))

best score: 0.83


In [31]:
# evaluate model
print('score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test,y_test)))

score for logistic regression - version 2: 0.83


In [32]:
# get submission file
get_submission_file(clf,'03_lr.csv')

## Feature Normalisation and Standardisation

In [33]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [34]:
# feature normalisation
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [35]:
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [36]:
# normalise test data
X_test_scaled = scaler.transform(X_test)

### Feature Standardisation

In [37]:
# feature standardisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Create model after standardisation

In [38]:
# base model
model_lr = LogisticRegression()
parameters = {'C':[1.0,10.0,50.0,100.0,1000.0],'penalty':['l1','l2']}
clf = GridSearchCV(model_lr,param_grid=parameters,cv=3)
clf.fit(X_train_scaled,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [39]:
clf.best_score_

0.8132022471910112

In [40]:
# evaluate model
print('score for logistic regression - ver 2: {0:.2f}'.format(clf.score(X_test_scaled,y_test)))

score for logistic regression - ver 2: 0.84


## Model Persistence

In [41]:
import pickle

In [43]:
# create file paths
model_file_path = os.path.join(os.path.pardir,'models','lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir,'models','lr_scaler.pkl')

In [44]:
# open files to write
model_file_pickle = open(model_file_path,'wb')
scaler_file_pickle = open(scaler_file_path,'wb')

In [45]:
# persist the model and scaled data
pickle.dump(clf,model_file_pickle)
pickle.dump(scaler,scaler_file_pickle)

In [46]:
# close the files
model_file_pickle.close()
scaler_file_pickle.close()

### load the presisted files

In [49]:
# open files
model_file_pickle = open(model_file_path,'rb')
scaler_file_pickle = open(scaler_file_path,'rb')
# load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
# close files
model_file_pickle.close()
scaler_file_pickle.close()

In [50]:
clf_loaded

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [52]:
# transform the test data with the laoded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
print('score for persisted logistic regression: {0:.2f}'.format(clf_loaded.score(X_test_scaled,y_test)))

score for persisted logistic regression: 0.84
